In [1]:
%config IPCompleter.greedy=True

In [2]:
import pandas as pd

df_train = pd.read_csv('train_kaggle.csv')
df_test = pd.read_csv('sample_solution.csv')
Y = df_train['Label'].values

In [3]:
import numpy as np

dataframes = np.load('allData.npy', allow_pickle = True)
dataframes.shape

def load_test_dataframe(id):
    test_data = np.load("test/test/{}.npy".format(id))
    return pd.DataFrame(data=test_data)

testdatas = []
for id in df_test['Id']:
    dfi = load_test_dataframe(id)
    testdatas.append(dfi.values)

In [4]:
# feat = dataframes[0].shape[1]
feat = 12

In [5]:
from sklearn.preprocessing import OneHotEncoder

def one_hot_encoding(encoder, dfs, col):
    if encoder == None:
        data = set(dfs[0][:,col])
        for i in range(1,len(dfs)):
            df = dfs[i]
            data |= set(df[:,col])
        encoder = OneHotEncoder(handle_unknown='ignore')
        encoder = encoder.fit([[str(e)] for e in data])
    encodedData = []
    for df in dfs:
        encodedData.append(np.sum(encoder.transform([[str(e)] for e in df[:,col]]), axis=0))
    encodedData = np.stack(encodedData)    
    return encoder, encodedData

In [21]:
from tqdm import tqdm

encoders = []
encoder, XTrain = one_hot_encoding(None, dataframes, 0)
encoders.append(encoder)
for i in tqdm(range(1, feat)):
    encoder, train = one_hot_encoding(None, dataframes, i)
    encoders.append(encoder)
    XTrain = np.concatenate((XTrain, train), axis=1)
XTrain.shape

100%|█████████████████████████████████████████████████████████| 11/11 [05:10<00:00, 28.20s/it]


(18662, 60)

In [22]:
encoder, XTest = one_hot_encoding(None, testdatas, 0)
for i in tqdm(range(1, feat)):
    encoder, test = one_hot_encoding(None, testdatas, i)
    encoders.append(encoder)
    XTest = np.concatenate((XTest, test), axis=1)
XTest.shape

100%|█████████████████████████████████████████████████████████| 11/11 [01:27<00:00,  7.94s/it]


(6051, 60)

In [47]:
XTrain = np.squeeze(np.asarray(XTrain))
XTest = np.squeeze(np.asarray(XTest))

In [48]:
np.save('1_2_features_train.npy', XTrain)
np.save('1_2_features_test.npy', XTest)

In [49]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(XTrain, Y, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(14929, 60)
(3733, 60)
(14929,)
(3733,)


In [50]:
X_train

array([[ 34.,   3.,   0., ...,   0.,   7.,   1.],
       [ 44.,  11.,   0., ...,   0.,  37.,   0.],
       [119.,   6.,   0., ...,   0.,  18.,   1.],
       ...,
       [ 32.,   3.,   0., ...,   0.,   3.,   1.],
       [ 34.,   3.,   0., ...,   0.,   7.,   1.],
       [ 32.,   3.,   0., ...,   0.,   3.,   1.]])

In [52]:
import lightgbm as lgb

train_set = lgb.Dataset(X_train.astype(np.int32), y_train.astype(int))
valid_set = lgb.Dataset(X_valid.astype(np.int32), y_valid.astype(int))

from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier

clf = ExtraTreesClassifier(n_estimators=50)
clf = clf.fit(XTrain, Y)
modelSelection = SelectFromModel(clf, prefit=True, max_features=8000)
XTrain = modelSelection.transform(XTrain)
XTrain.shape

XTest = modelSelection.transform(XTest)
XTest.shape

In [53]:
from hyperopt import hp
from hyperopt import Trials
from hyperopt import fmin
from hyperopt import tpe
import lightgbm as lgb
from hyperopt import STATUS_OK

ITER = 50
STOP_ROUND = 5

# Create the dataset
def objective(params):
    """Objective function for Gradient Boosting Machine Hyperparameter Tuning"""
    params['num_leaves'] = int(params['num_leaves'])
    params['subsample_for_bin'] = int(params['subsample_for_bin'])
    params['min_child_samples'] = int(params['min_child_samples'])
    # Perform n_fold cross validation with hyperparameters
    # Use early stopping and evalute based on ROC AUC
    bst = lgb.train(params, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
    bst.save_model('model.txt', num_iteration=bst.best_iteration)
  
    # Extract the best score
    best_score = bst.best_score['valid_0']['auc']
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

# Define the search space
space = {
    'boosting_type': hp.choice('boosting_type', 
                                [
                                    'gbdt',
                                    'dart',
                                    'goss'
                                ]),
    'num_leaves': hp.quniform('num_leaves', 30, 150, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.01), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 500, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 1.0),
    'metric': 'auc'
}

# Trials object to track progress
bayes_trials = Trials()

MAX_EVALS = 500

# Optimize
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials)
best

[1]	valid_0's auc: 0.948941                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.955375                                                                   
[3]	valid_0's auc: 0.959766                                                                   
[4]	valid_0's auc: 0.959239                                                                   
[5]	valid_0's auc: 0.960317                                                                   
[6]	valid_0's auc: 0.962078                                                                   
[7]	valid_0's auc: 0.962497                                                                   
[8]	valid_0's auc: 0.962269                                                                   
[9]	valid_0's auc: 0.962109                                                                   
[10]	valid_0's auc: 0.962831                      

[4]	valid_0's auc: 0.957817                                                                   
[5]	valid_0's auc: 0.957489                                                                   
[6]	valid_0's auc: 0.96036                                                                    
[7]	valid_0's auc: 0.961701                                                                   
[8]	valid_0's auc: 0.961781                                                                   
[9]	valid_0's auc: 0.961512                                                                   
[10]	valid_0's auc: 0.962558                                                                  
[11]	valid_0's auc: 0.962588                                                                  
[12]	valid_0's auc: 0.962871                                                                  
[13]	valid_0's auc: 0.963661                                                                  
[14]	valid_0's auc: 0.96352                       

[3]	valid_0's auc: 0.964154                                                                   
[4]	valid_0's auc: 0.965781                                                                   
[5]	valid_0's auc: 0.966342                                                                   
[6]	valid_0's auc: 0.967263                                                                   
[7]	valid_0's auc: 0.968224                                                                   
[8]	valid_0's auc: 0.96823                                                                    
[9]	valid_0's auc: 0.968009                                                                   
[10]	valid_0's auc: 0.968225                                                                  
[11]	valid_0's auc: 0.968596                                                                  
[12]	valid_0's auc: 0.968489                                                                  
[13]	valid_0's auc: 0.968906                      

[17]	valid_0's auc: 0.969126                                                                  
[18]	valid_0's auc: 0.969225                                                                  
[19]	valid_0's auc: 0.969377                                                                  
[20]	valid_0's auc: 0.969725                                                                  
[21]	valid_0's auc: 0.970085                                                                  
[22]	valid_0's auc: 0.970178                                                                  
[23]	valid_0's auc: 0.970257                                                                  
[24]	valid_0's auc: 0.970744                                                                  
[25]	valid_0's auc: 0.971165                                                                  
[26]	valid_0's auc: 0.971309                                                                  
[27]	valid_0's auc: 0.971418                      

[38]	valid_0's auc: 0.964475                                                                  
[39]	valid_0's auc: 0.964821                                                                  
[40]	valid_0's auc: 0.964888                                                                  
[41]	valid_0's auc: 0.965202                                                                  
[42]	valid_0's auc: 0.965404                                                                  
[43]	valid_0's auc: 0.965579                                                                  
[44]	valid_0's auc: 0.965749                                                                  
[45]	valid_0's auc: 0.965973                                                                  
[46]	valid_0's auc: 0.96612                                                                   
[47]	valid_0's auc: 0.966384                                                                  
[48]	valid_0's auc: 0.966498                      

...

[5]	valid_0's auc: 0.974882                                                                   
[6]	valid_0's auc: 0.913321                                                                   
[7]	valid_0's auc: 0.366015                                                                   
[8]	valid_0's auc: 0.169968                                                                   
[9]	valid_0's auc: 0.245063                                                                   
[10]	valid_0's auc: 0.276361                                                                  
Early stopping, best iteration is:                                                            
[5]	valid_0's auc: 0.974882
[1]	valid_0's auc: 0.961427                                                                   
Training until validation scores don't improve for 5 rounds                                   
[2]	valid_0's auc: 0.971128                                                                   
[3]	valid_0's auc: 0.9

[26]	valid_0's auc: 0.97076                                                                   
[27]	valid_0's auc: 0.970738                                                                  
[28]	valid_0's auc: 0.970779                                                                  
[29]	valid_0's auc: 0.970986                                                                  
[30]	valid_0's auc: 0.971042                                                                  
[31]	valid_0's auc: 0.971094                                                                  
[32]	valid_0's auc: 0.971065                                                                  
[33]	valid_0's auc: 0.971632                                                                  
[34]	valid_0's auc: 0.971761                                                                  
[35]	valid_0's auc: 0.971845                                                                  
[36]	valid_0's auc: 0.971982                      

[7]	valid_0's auc: 0.967195                                                                   
[8]	valid_0's auc: 0.968211                                                                   
[9]	valid_0's auc: 0.969265                                                                   
[10]	valid_0's auc: 0.970234                                                                  
[11]	valid_0's auc: 0.970737                                                                  
[12]	valid_0's auc: 0.972159                                                                  
[13]	valid_0's auc: 0.972745                                                                  
[14]	valid_0's auc: 0.973841                                                                  
[15]	valid_0's auc: 0.974555                                                                  
[16]	valid_0's auc: 0.974829                                                                  
[17]	valid_0's auc: 0.975381                      

[40]	valid_0's auc: 0.987402                                                                  
[41]	valid_0's auc: 0.987465                                                                  
[42]	valid_0's auc: 0.987299                                                                  
[43]	valid_0's auc: 0.987381                                                                  
[44]	valid_0's auc: 0.987458                                                                  
[45]	valid_0's auc: 0.987496                                                                  
[46]	valid_0's auc: 0.987555                                                                  
[47]	valid_0's auc: 0.987607                                                                  
[48]	valid_0's auc: 0.98773                                                                   
[49]	valid_0's auc: 0.987803                                                                  
[50]	valid_0's auc: 0.987864                      

{'boosting_type': 0,
 'colsample_by_tree': 0.970001036839981,
 'learning_rate': 0.1293814508517374,
 'min_child_samples': 20.0,
 'num_leaves': 107.0,
 'reg_alpha': 0.0668379271957277,
 'reg_lambda': 0.48617229984565924,
 'subsample_for_bin': 240000.0}

In [62]:
best['boosting_type'] = 'dart'
best['num_leaves'] = int(best['num_leaves'])
best['subsample_for_bin'] = int(best['subsample_for_bin'])
best['min_child_samples'] = int(best['min_child_samples'])
lgbModel = lgb.train(best, train_set, ITER, valid_sets=valid_set, early_stopping_rounds=STOP_ROUND)
lgbModel.save_model('model.txt', num_iteration=bst.best_iteration)
YTest = lgbModel.predict(XTest)

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\callback.py:192: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


In [18]:
from lightgbm import LGBMClassifier
model = LGBMClassifier()
model.fit(XTrain, Y)
probs = model.predict_proba(XTest)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [63]:
from lightgbm import LGBMClassifier
model = LGBMClassifier(boosting_type=best['boosting_type'],
                        num_leaves=best['num_leaves'],
                        learning_rate=best['learning_rate'],
                        subsample_for_bin=best['subsample_for_bin'],
                        min_child_samples=best['min_child_samples'],
                        reg_alpha=best['reg_alpha'],
                        reg_lambda=best['reg_lambda'],
                        colsample_bytree=best['colsample_by_tree'])
model.fit(XTrain, Y)
probs = model.predict_proba(XTest, num_iteration=lgbModel.best_iteration)

In [64]:
YTest = probs[:, 0]

In [65]:
df_test['Predicted'] = YTest
df_test

,Id,Predicted
0,0,0.003121
1,1,0.015779
2,2,0.995651
3,3,0.432073
4,4,0.489502
5,5,0.941459
6,6,0.996613
7,7,0.628204
8,8,0.003121
9,9,0.006326


In [66]:
df_test.to_csv('test.csv', index=False)